In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import eda

from tqdm import tqdm

# os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id']

feat_no = '101_new_'

### csvをpickleにする

In [2]:
is_pickle = True
is_pickle = False

def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')

if is_pickle:
    new.sort_values(by=[key, 'purchase_date'], inplace=True)
    utils.to_df_pkl(path='../input', fname=f"new_merchant_transactions", df=new)

### Data Load

In [3]:
train = elo_load_data('train')
test = elo_load_data('test')
train_id = train[key].values
test_id = test[key].values
train_test = pd.concat([train, test], axis=0).reset_index(drop=True)
new = elo_load_data('new_')
# historical = elo_load_data('historical')

100%|██████████| 3/3 [00:00<00:00,  3.48it/s]


In [85]:
display(train.head())
print(train[key].drop_duplicates().shape)
print(train.shape)
display(new.head())
print(new.shape)
print(new[key].drop_duplicates().shape)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


(201917,)
(201917, 6)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1793298,Y,C_ID_00007093c1,76,N,1.0,B,222,M_ID_08f01305af,2,-0.671775,2018-04-03 11:13:35,3,2,21
1793299,Y,C_ID_00007093c1,69,N,1.0,B,879,M_ID_00a6ca8a8a,2,-0.656749,2018-04-09 16:23:59,1,9,29
38630,Y,C_ID_0001238066,314,N,1.0,B,307,M_ID_7d8102bb34,1,-0.732783,2018-03-01 16:48:27,1,9,19
38618,Y,C_ID_0001238066,314,N,1.0,B,367,M_ID_235e546dcc,1,-0.672136,2018-03-03 22:44:57,1,9,16
38633,Y,C_ID_0001238066,333,N,1.0,B,783,M_ID_a88790a464,1,-0.641722,2018-03-04 13:05:16,1,9,19


(1963031, 14)
(290001,)


#### New Transactionsの最少粒度は下記

In [118]:
print(new.shape)
cols = [key, 'purchase_date', 'purchase_amount', 'merchant_id', 'merchant_category_id', 'installments']
print(new[cols].drop_duplicates().shape)

(1963031, 14)
(1963031, 6)


SystemExit: 

### Cleansing

In [4]:
# 999という値が1つだけあるので欠損値扱いで-1にする
new['installments'] = new['installments'].where(new['installments']<20, -1)
new['installments'] = new['installments'].replace(-1, np.nan)
new['authorized_flag'] = new['authorized_flag'].replace(-1, 'NA')
new['merchant_id'] =  new['merchant_id'].fillna('NA')
new['category_2'] =  new['category_2'].fillna('NA')
new['category_3'] =  new['category_3'].fillna('NA')

### card_idの粒度で色々集計する
有効かわからないからとりあえず書いてやってみる

### nunique

In [5]:
feat_nunique = []
nuq_city_id = new.groupby(key)['city_id'].nunique().rename('city_id_nuq@card_id')
feat_nunique.append(nuq_city_id)

nuq_merchant_category_id = new.groupby(key)['merchant_category_id'].nunique().rename('merchant_category_id_nuq@card_id')
feat_nunique.append(nuq_merchant_category_id)

nuq_merchant_id = new.groupby(key)['merchant_id'].nunique().rename('merchant_id_nuq@card_id')
feat_nunique.append(nuq_merchant_id)

nuq_state_id = new.groupby(key)['state_id'].nunique().rename('state_id_nuq@card_id')
feat_nunique.append(nuq_state_id)

nuq_subsector_id = new.groupby(key)['subsector_id'].nunique().rename('subsector_id_nuq@card_id')
feat_nunique.append(nuq_subsector_id)

nuq_category_1 = new.groupby(key)['category_1'].nunique().rename('category_1_nuq@card_id')
feat_nunique.append(nuq_category_1)

nuq_category_2 = new.groupby(key)['category_2'].nunique().rename('category_2_nuq@card_id')
feat_nunique.append(nuq_category_2)

nuq_category_3 = new.groupby(key)['category_3'].nunique().rename('category_3_nuq@card_id')
feat_nunique.append(nuq_category_3)

nuq_installments = new.groupby(key)['installments'].nunique().rename('installments_nuq@card_id')
feat_nunique.append(nuq_installments)

nuq_purchase_date = new.groupby(key)['purchase_date'].nunique().rename('purchase_date_nuq@card_id')
feat_nunique.append(nuq_purchase_date)

nuq_month_lag = new.groupby(key)['month_lag'].nunique().rename('month_lag_nuq@card_id')
feat_nunique.append(nuq_month_lag)

### month_lag別のnunique

In [6]:
lag_1 = new.query("month_lag==1")
lag_2 = new.query("month_lag==2")
feat_month_lag = []

# lag_1
cnt_city_id = lag_1.groupby(key)['city_id'].nunique().rename('city_id_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_city_id)

cnt_merchant_category_id = lag_1.groupby(key)['merchant_category_id'].nunique().rename('merchant_category_id_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_merchant_category_id)

cnt_merchant_id = lag_1.groupby(key)['merchant_id'].nunique().rename('merchant_id_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_merchant_id)

cnt_state_id = lag_1.groupby(key)['state_id'].nunique().rename('state_id_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_state_id)

cnt_subsector_id = lag_1.groupby(key)['subsector_id'].nunique().rename('subsector_id_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_subsector_id)

cnt_category_1 = lag_1.groupby(key)['category_1'].nunique().rename('category_1_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_category_1)

cnt_category_2 = lag_1.groupby(key)['category_2'].nunique().rename('category_2_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_category_2)

cnt_category_3 = lag_1.groupby(key)['category_3'].nunique().rename('category_3_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_category_3)

cnt_installments = lag_1.groupby(key)['installments'].nunique().rename('installments_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_installments)

cnt_purchase_date = lag_1.groupby(key)['purchase_date'].nunique().rename('purchase_date_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_purchase_date)

cnt_month_lag = lag_1.groupby(key)['month_lag'].nunique().rename('month_lag_nuq@card_id-month_lag1')
feat_month_lag.append(cnt_month_lag)

# lag_2
cnt_city_id = lag_2.groupby(key)['city_id'].nunique().rename('city_id_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_city_id)

cnt_merchant_category_id = lag_2.groupby(key)['merchant_category_id'].nunique().rename('merchant_category_id_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_merchant_category_id)

cnt_merchant_id = lag_2.groupby(key)['merchant_id'].nunique().rename('merchant_id_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_merchant_id)

cnt_state_id = lag_2.groupby(key)['state_id'].nunique().rename('state_id_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_state_id)

cnt_subsector_id = lag_2.groupby(key)['subsector_id'].nunique().rename('subsector_id_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_subsector_id)

cnt_category_1 = lag_2.groupby(key)['category_1'].nunique().rename('category_1_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_category_1)

cnt_category_2 = lag_2.groupby(key)['category_2'].nunique().rename('category_2_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_category_2)

cnt_category_3 = lag_2.groupby(key)['category_3'].nunique().rename('category_3_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_category_3)

cnt_installments = lag_2.groupby(key)['installments'].nunique().rename('installments_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_installments)

cnt_purchase_date = lag_2.groupby(key)['purchase_date'].nunique().rename('purchase_date_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_purchase_date)

cnt_month_lag = lag_2.groupby(key)['month_lag'].nunique().rename('month_lag_nuq@card_id-month_lag2')
feat_month_lag.append(cnt_month_lag)

### 作成したnunique featureをjoin

In [7]:
train_test.set_index(key, inplace=True)
for feat in feat_nunique:
    train_test = train_test.join(feat, how='left')
    
for feat in feat_month_lag:
    train_test = train_test.join(feat, how='left')

### month_lag別の集計

In [8]:
feat_ins = new.groupby(key)['installments'].agg({
    'installments_max@card_id':'max'
    ,'installments_min@card_id':'min'
    ,'installments_mean@card_id':'mean'
    ,'installments_std@card_id':'std'
    ,'installments_skew@card_id':'skew'
})

feat_amt = new.groupby(key)['purchase_amount'].agg({
    'purchase_amount_max@card_id':'max'
    ,'purchase_amount_min@card_id':'min'
    ,'purchase_amount_mean@card_id':'mean'
    ,'purchase_amount_std@card_id':'std'
    ,'purchase_amount_skew@card_id':'skew'
})

# lag_1
feat_lag1_ins = lag_1.groupby(key)['installments'].agg({
    'installments_max@card_id-lag1':'max'
    ,'installments_min@card_id-lag1':'min'
    ,'installments_mean@card_id-lag1':'mean'
    ,'installments_std@card_id-lag1':'std'
    ,'installments_skew@card_id-lag1':'skew'
})

feat_lag1_amt = lag_1.groupby(key)['purchase_amount'].agg({
    'purchase_amount_max@card_id-lag1':'max'
    ,'purchase_amount_min@card_id-lag1':'min'
    ,'purchase_amount_mean@card_id-lag1':'mean'
    ,'purchase_amount_std@card_id-lag1':'std'
    ,'purchase_amount_skew@card_id-lag1':'skew'
})

# lag_2
feat_lag2_ins = lag_2.groupby(key)['installments'].agg({
    'installments_max@card_id-lag2':'max'
    ,'installments_min@card_id-lag2':'min'
    ,'installments_mean@card_id-lag2':'mean'
    ,'installments_std@card_id-lag2':'std'
    ,'installments_skew@card_id-lag2':'skew'
})

feat_lag2_amt = lag_2.groupby(key)['purchase_amount'].agg({
    'purchase_amount_max@card_id-lag2':'max'
    ,'purchase_amount_min@card_id-lag2':'min'
    ,'purchase_amount_mean@card_id-lag2':'mean'
    ,'purchase_amount_std@card_id-lag2':'std'
    ,'purchase_amount_skew@card_id-lag2':'skew'
})

### aggregate feature setをjoin

In [9]:
for feat in [feat_ins, feat_amt, feat_lag1_ins, feat_lag1_amt, feat_lag2_ins, feat_lag2_amt]:
    train_test = train_test.join(feat, how='left')

### New Transactionsで作成した, nunique feature/lag1, lag2 nunique fewture/date_diff, date_ratio featureをjoin

In [10]:
train = train_test.loc[train_id, :]
test = train_test.loc[test_id, :]
for col in train.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = train[col].values, path=f'../features/1_first_valid/{feat_no}train_{col}')
        utils.to_pkl_gzip(obj = test[col].values, path=f'../features/1_first_valid/{feat_no}test_{col}')